In [12]:
import numpy as np 
import matplotlib as plt 
import os 
import time
import cv2 
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers,models,activations
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Input,LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

## Extract keypoints using media pipe hands

In [3]:
#code derived from https://google.github.io/mediapipe/solutions/hands.html
#mediapipe models
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles

# detect landmarks/keypoints on hands 
def mp_detect(image, model):
    image.flags.writeable = False 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

# draw landmarks on hands
def mp_draw_landmarks(image, results):
    image.flags.writeable = True 
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                      mp_drawing_styles.get_default_hand_landmarks_style(),
                                      mp_drawing_styles.get_default_hand_connections_style())
            


In [5]:
# testing hand recongition 
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    
    while cap.isOpened():
        # capture frame 
        retval, image = cap.read()
        
        # detect landmarks/keypoints 
        image, results = mp_detect(image,hands)
        
        # draw landmarks/keypoints 
        mp_draw_landmarks(image,results)
        
        # visualise
        cv2.imshow('OpenCV Feed', image)
        # quit with q press 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()


In [4]:
# extract keypoints 
# 21 keypoints for each hand on the screen 
def extract_keypoints(results):    
    if results.multi_hand_landmarks:
        h1 = np.array([[lm.x, lm.y, lm.z] 
                       for lm in results.multi_hand_landmarks[0].landmark]).flatten()
        # check for second hand 
        if len(results.multi_hand_landmarks) == 2:
            h2 = np.array([[lm.x, lm.y, lm.z] 
                   for lm in results.multi_hand_landmarks[1].landmark]).flatten()
        else:
            h2 = np.zeros(21*3)
        return np.concatenate([h1,h2])
    else: 
        return np.zeros(42*3)
    

## Collect Training and Testing Data 

In [5]:
gestures = np.array(['play','pause','nothing'])
no_datapoints = 30
no_frames = 30
DATA_PATH = os.path.join(os.getcwd(),"Gestures_Data")

In [7]:
#create folders for each datapoint collected from videos 
for gesture in gestures:
    for datapoint in range (1,no_datapoints+1):
        try:
            os.makedirs(os.path.join(DATA_PATH,gesture,str(datapoint)))
        except:
            pass
    

In [8]:
# collect and save video data of the gestures as np arrays, 30 frames per video
font = cv2.FONT_HERSHEY_SIMPLEX
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.5) as hands:
    for gesture in gestures:
        for datapoint in range(1,no_datapoints+1): 
            for frame_num in range(no_frames):
                
                # capture frame 
                retval, image = cap.read()

                # detect landmarks/keypoints 
                image, results = mp_detect(image,hands)
                
                # draw landmarks/keypoints
                mp_draw_landmarks(image,results)
                
                # show text at the start and end of the recording 
                # start of recording 
                if frame_num == 0:
                    cv2.putText(image,'START',(250,260),font,2,(0,255,0),2, cv2.LINE_AA)
                    cv2.putText(image,'Gesture: {} || Video: {}'.format(gesture,str(datapoint)),(20, 20),
                               font,0.5,(0,255,0),1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    # wait 2 second 
                    cv2.waitKey(500)
                # end of recording 
                elif frame_num == no_frames-1:
                    cv2.putText(image,'END',(250,260),font,2,(0,255,0),2, cv2.LINE_AA)
                    cv2.putText(image,'Gesture: {} || Video: {}'.format(gesture,str(datapoint)),(20, 20),
                               font,0.5,(0,255,0),1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    # wait 2 seconds
                    cv2.waitKey(2000)

                else:
                    cv2.putText(image,'Gesture: {} || Video: {}'.format(gesture,str(datapoint)),(20, 20),
                               font,0.5,(0,255,0),1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    
                # extract and save keypoints for each frame
                keypoints = extract_keypoints(results)
                path = os.path.join(DATA_PATH,gesture,str(datapoint),str(frame_num))
                np.save(path,keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

## Preprocess data 

In [9]:
def load_data(gesture,datapoint):
    label = gesture
    data = []
    for frame_num in range(no_frames):
        frame = np.load(os.path.join(DATA_PATH, gesture, str(datapoint),"{}.npy".format(frame_num)))
        data.append(frame)
    return data, label       


In [10]:
labels_map = {label:num for num, label in enumerate(gestures)}
datapoints, labels = [], []
for gesture in gestures:
    for datapoint in range(1,no_datapoints+1):
        data, label = load_data(gesture,datapoint)
        datapoints.append(data)
        labels.append(labels_map[gesture])

In [11]:
X = np.array(datapoints)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)


## Create and Train Basic LSTM Model 

In [12]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [14]:
inputs = Input((30,126))
lstm = LSTM(64,return_sequences=True, activation = 'relu',input_shape=(30,126))(inputs)
lstm2 = LSTM(128, activation = 'relu')(lstm)
de = Dense(128, activation='relu')(lstm2)
de2 = Dense(64, activation='relu')(de)
outputs = Dense(len(gestures),activation='softmax')(de2)
model = models.Model(inputs,outputs)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30, 126)]         0         
_________________________________________________________________
lstm (LSTM)                  (None, 30, 64)            48896     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 172,675
Trainable params: 172,675
Non-trainable params: 0
_______________________________________________________

In [15]:
model.fit(X_train, y_train, epochs=250, callbacks=[tb_callback])

Epoch 1/250
3/3 [==============================] - 4s 550ms/step - loss: 1.0826 - categorical_accuracy: 0.4074
Epoch 2/250
3/3 [==============================] - 1s 251ms/step - loss: 1.0092 - categorical_accuracy: 0.3333
Epoch 3/250
3/3 [==============================] - 1s 250ms/step - loss: 0.8652 - categorical_accuracy: 0.3086
Epoch 4/250
3/3 [==============================] - 1s 257ms/step - loss: 0.8085 - categorical_accuracy: 0.7037
Epoch 5/250
3/3 [==============================] - 1s 269ms/step - loss: 1.0107 - categorical_accuracy: 0.8148
Epoch 6/250
3/3 [==============================] - 1s 253ms/step - loss: 0.7521 - categorical_accuracy: 0.8272
Epoch 7/250
3/3 [==============================] - 1s 248ms/step - loss: 0.8098 - categorical_accuracy: 0.8519
Epoch 8/250
3/3 [==============================] - 1s 254ms/step - loss: 0.8236 - categorical_accuracy: 0.8272
Epoch 9/250
3/3 [==============================] - 1s 241ms/step - loss: 0.7614 - categorical_accuracy: 0.8519
E

3/3 [==============================] - 1s 249ms/step - loss: 0.1594 - categorical_accuracy: 0.9630
Epoch 147/250
3/3 [==============================] - 1s 260ms/step - loss: 0.1708 - categorical_accuracy: 0.9259
Epoch 148/250
3/3 [==============================] - 1s 247ms/step - loss: 0.1808 - categorical_accuracy: 0.9259
Epoch 149/250
3/3 [==============================] - 1s 243ms/step - loss: 0.1622 - categorical_accuracy: 0.9506
Epoch 150/250
3/3 [==============================] - 1s 248ms/step - loss: 0.1556 - categorical_accuracy: 0.9630
Epoch 151/250
3/3 [==============================] - 1s 246ms/step - loss: 0.1532 - categorical_accuracy: 0.9877
Epoch 152/250
3/3 [==============================] - 1s 258ms/step - loss: 0.1442 - categorical_accuracy: 0.9753
Epoch 153/250
3/3 [==============================] - 1s 256ms/step - loss: 0.1352 - categorical_accuracy: 0.9753
Epoch 154/250
3/3 [==============================] - 1s 251ms/step - loss: 0.1253 - categorical_accuracy: 0.97

In [16]:
#save model
model.save('gestures.h5')

In [10]:
#load model
model = models.load_model('gestures.h5')

In [11]:
frames= []
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    
    while cap.isOpened():
        # capture frame 
        retval, image = cap.read()
        
        # detect landmarks/keypoints 
        image, results = mp_detect(image,hands)
        
        # draw landmarks/keypoints 
        mp_draw_landmarks(image,results)
        
        #extract keypoints
        keypoints = extract_keypoints(results)
        
        #prediction
        frames.append(keypoints)
        frames = frames[-30:]
        if len(frames) == 30:
            pred = model.predict(np.expand_dims(frames,axis=0))[0]
            print(gestures[np.argmax(pred)])
        
        # visualise
        cv2.imshow('OpenCV Feed', image)
        # quit with q press 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
nothing
play
play
play
play
play
play
play
play
play
play
play
pause
pause
pause
pause
pause
play
play
play
play
play
play
play
play
play
play
play
play
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause
pause